<a href="https://colab.research.google.com/github/mithunkumarsr/CodeMixingDravidianLanguage/blob/main/BERTCMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 105 kB/s 
     |████████████████████████████████| 6.8 MB 18.9 MB/s 
     |████████████████████████████████| 981 kB 59.7 MB/s 
     |████████████████████████████████| 263 kB 59.1 MB/s 
     |████████████████████████████████| 1.9 MB 66.0 MB/s 
     |████████████████████████████████| 1.2 MB 43.0 MB/s 
     |████████████████████████████████| 468 kB 68.0 MB/s 
     |████████████████████████████████| 895 kB 67.5 MB/s 
     |████████████████████████████████| 3.3 MB 47.1 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.3-py3-none-any.whl size=25283131 sha256=0d31f11524ff465b8e3ee17f02098a313832a0677a6f1306829c13c0763323b1
  Stored in directory: /root/.cache/pip/wheels/3c/ef/e1/7da805d8a5944e8a3ac0553831d832c00b1800b848939849cb
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=31b138b1591399fa1cd3c2b8755982e96746f08d14e42aa2bf4d1d0f90633174
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff

In [ ]:
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords
import spacy
from sklearn.metrics import accuracy_score,classification_report, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
import ktrain
from ktrain import text

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [ ]:
# DATA CLEANING AND PREPARATION #
class Utils(object):

    def cleanText(self, text):
        review = re.sub(r"^https://t.co/[a-zA-Z0-9]*\s", " ", str(text))
        review = re.sub(r"\([\s\S]*\)", " ", str(review))
        review = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*\s", " ", str(review))
        review = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*$", " ", str(review))
        review = review.lower()
        review = re.sub(r"that's", "that is", str(review))
        review = re.sub(r"there's", "there is", str(review))
        review = re.sub(r"what's", "what is", str(review))
        review = re.sub(r"where's", "where is", str(review))
        review = re.sub(r"it's", "it is", str(review))
        review = re.sub(r"who's", "who is", str(review))
        review = re.sub(r"i'm", "i am", str(review))
        review = re.sub(r"she's", "she is", str(review))
        review = re.sub(r"he's", "he is", str(review))
        review = re.sub(r"they're", "they are", str(review))
        review = re.sub(r"who're", "who are", str(review))
        review = re.sub(r"ain't", "am not", str(review))
        review = re.sub(r"wouldn't", "would not", str(review))
        review = re.sub(r"shouldn't", "should not", str(review))
        review = re.sub(r"can't", "can not", str(review))
        review = re.sub(r"couldn't", "could not", str(review))
        review = re.sub(r"won't", "will not", str(review))
        review = re.sub(r" pm ", " ", str(review))
        review = re.sub(r" am ", " ", str(review))
        review = re.sub(r'[^\[\]]+(?=\])', " ", str(review))
        review = re.sub(r"\W", " ", str(review))
        review = re.sub(r"\d", " ", str(review))
        review = re.sub(r"\s+[a-z]\s+", " ", str(review))
        review = re.sub(r"\s+[a-z]$", " ", str(review))
        review = re.sub(r"^[a-z]\s+", " ", str(review))
        review = re.sub(r"\s+", " ", str(review))
        return review

    def remove_punc(self, text):
        table = str.maketrans("", "", string.punctuation)
        return text.translate(table)

    def remove_emoticon(self, text):
        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)


    
    def lemmatization(self, text):
        doc = nlp(text)
        return " ".join([token.lemma_ for token in doc])

    nltk.download('stopwords')
    def remove_stops(self, text):
        stop_words = set(nltk.corpus.stopwords.words('english'))
        text = [word.lower() for word in text.split() if word.lower() not in stop_words]
        return " ".join(text)


    def readData1(self, path, inputColumnIndex=0, outputColumnIndex=1):
        df = pd.read_csv(path, error_bad_lines=False, sep='\t')
        X = df.iloc[:, inputColumnIndex].values
        y = df.iloc[:, outputColumnIndex].values
        return X,y
    
    def readData2(self, path, inputColumnIndex=1, outputColumnIndex=2):
        df = pd.read_csv(path, error_bad_lines=False, sep='\t')
        X = df.iloc[:, inputColumnIndex].values
        y = df.iloc[:, outputColumnIndex].values
        return X,y
    
    def draw_prediction_results(self, y_pred, y_test, my_tags, method):
        print('accuracy of ' + method + ': %s' % accuracy_score(y_pred, y_test))
        print(classification_report(y_test, y_pred, target_names=my_tags, digits = 6))

    
    def crossValidation(self, prediction, input, output, k=5):
        scores = cross_val_score(prediction, input,output, cv=k)
        print("Accuracy of Cross Validation Mean: %0.6f (+/- %0.6f)" % (scores.mean(), scores.std() * 2))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data_train = pd.read_csv('tamil_sentiment_full_train.tsv', error_bad_lines=False, sep='\t')
data_train

,text,category
0,Vani bhojam fans hit like solli 500 like Vangi...,unknown_state
1,I love you ajith very I like,Positive
2,ennaya trailer Ku mudi Ellam nikkudhu... Vera ...,Positive
3,Vijay Annaa Ur Maassssss Therrrrriiiiii,Positive
4,நம்ப நடே நாசாமா தான் போச்சி,Negative
...,...,...
35651,ஒருவர் செய்த தவறுக்காக எல்லாரையும் பழி போடுவது...,Mixed_feelings
35652,Repeated Mode to Watching Theri Trailer & Ther...,Positive
35653,yeevanikuachu pula kutti erudhal oodi poyidu....,Negative
35654,She is looking like laughing budha,Positive


In [ ]:
data_test = pd.read_csv('tamil_sentiment_full_test_withtlabels.tsv', error_bad_lines=False, sep='\t')
data_test

,id,text,category
0,Tam_1,வீர செங்குந்தர் சார்பாக இந்த திரைப்படம் வெற்றி...,Positive
1,Tam_2,Teruk ah irukku .... mokke movie .. waste of time,Negative
2,Tam_3,manitha samuthaayam amaipil irunthu intha pada...,Positive
3,Tam_4,JJ mam we miss u,Positive
4,Tam_5,Subtitle me traller dekhne wale like karo,not-Tamil
...,...,...,...
4397,Tam_4398,Ithukum dislike potta kammanattti koovaingalam...,Negative
4398,Tam_4399,Suyama Sinthikiravan than super Hero Seama dia...,Mixed_feelings
4399,Tam_4400,Super thalaiva.... Nee mass dha eppavume,Positive
4400,Tam_4401,பெண்ணை அடிமையாக்க நினைக்கும் இந்த படம் தோல்வித...,unknown_state


In [ ]:
 (X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(train_df = data_train,
                   text_column = 'text',
                   label_columns = 'category',
                   val_df = data_test,
                   maxlen = 150,
                   preprocess_mode = 'bert')

['Mixed_feelings', 'Negative', 'Positive', 'Positive ', 'not-Tamil', 'unknown_state']
   Mixed_feelings  Negative  Positive  Positive   not-Tamil  unknown_state
0             0.0       0.0       0.0        0.0        0.0            1.0
1             0.0       0.0       1.0        0.0        0.0            0.0
2             0.0       0.0       1.0        0.0        0.0            0.0
3             0.0       0.0       1.0        0.0        0.0            0.0
4             0.0       1.0       0.0        0.0        0.0            0.0
['Mixed_feelings', 'Negative', 'Positive', 'Positive ', 'not-Tamil', 'unknown_state']
   Mixed_feelings  Negative  Positive  Positive   not-Tamil  unknown_state
0             0.0       0.0       1.0        0.0        0.0            0.0
1             0.0       1.0       0.0        0.0        0.0            0.0
2             0.0       0.0       1.0        0.0        0.0            0.0
3             0.0       0.0       1.0        0.0        0.0            0.0
4  

Is Multi-Label? False
preprocessing test...
language: ta


In [ ]:
X_train[0].shape

(35656, 150)

In [ ]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 150
done.


In [ ]:
# Get Learning Rate
learner = ktrain.get_learner(model = model,
                             train_data = (X_train, y_train),
                             val_data = (X_test, y_test),
                             batch_size = 6)

In [ ]:
#Takes lot of time to train
'''
learner.lr_find()
learner.lr_plot()
'''
#Optimal learning rate for this model is 2e-5

'\nlearner.lr_find()\nlearner.lr_plot()\n'

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
 795/5943 [===>..........................] - ETA: 15:50:10 - loss: 1.2530 - accuracy: 0.5421